In [10]:
import os, json, math
from typing import Dict, List, Tuple
from concurrent.futures import ProcessPoolExecutor, as_completed

import xml.etree.ElementTree as ET
from xml.dom import minidom
import openslide
from openslide.deepzoom import DeepZoomGenerator
import os, json, math
from typing import List, Dict, Tuple, Optional

from draw_utils import *

# # DeepZoom 설정 (모델/추론 파이프라인 기본이 224일 가능성 높음)
# TILE_SIZE = 224
# OVERLAP   = 0

In [13]:
output_dir = "/home/sjhong/segmentation/histoplus/output/1-028_08" #1_152_19
os.makedirs(output_dir, exist_ok=True)
wsi_path = "/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-028_08.svs"
wsi_name = wsi_path.split('/')[-1][:-4]
iiss = (588, 602, 714)
ix = 0
json_path = f"/home/sjhong/segmentation/histoplus/output/KBSMC_HEATMAP_test/seg_{wsi_name}_{iiss[ix]}.json"
if not os.path.exists(json_path):
    
    json_path = f"/home/sjhong/segmentation/histoplus/output/1_028_08/seg_{wsi_name}_{iiss[ix]}.json"

os.path.exists(json_path)

True

In [14]:
# JSON 로딩(여러 파일이면 순차 로딩 + 레코드 합치기)
with (open(json_path, "r", encoding="utf-8") if not json_path.endswith(".gz") else __import__("gzip").open(json_path, "rt", encoding="utf-8")) as f:
    data = json.load(f, parse_float=float)

In [4]:
#<!--ROI: x0=72931, y0=28727, width=8465, height=2604-->

In [15]:
'''
x0 = 49142
y0 = 42066
x1 = 56092
y1 = 46847
w = x1-x0
h= y1-y0
w, h
'''

x0 = 20450
y0 = 43558
x1 = 31436
y1 = 56318
w = x1-x0
h= y1-y0
w, h

(10986, 12760)

In [16]:
# ── 사용 예 ───────────────────────────────────────────────────
slide = openslide.OpenSlide(wsi_path)
dz = DeepZoomGenerator(slide, tile_size=224, overlap=0, limit_bounds=False)
lvl = 16
# x0, y0 = 72931, 28727 # 좌측 상단 좌표
# w, h   = 8465, 2604

# (A) 낱개 타일 리스트
tiles = tiles_covering_base_rect(slide, dz, lvl, x0, y0, w, h,
                                 tile_size=224, overlap=0, return_spans=False)
if len(tiles) == 1:
    print("I founded this tile:",tiles[0])
    tile = tiles[0]
else:
    print(f"tile count: {len(tiles)}")
    print("first 10:", tiles[:10])  # (level, tx, ty)

'''
    # (B) 행 단위 span (더 빠르게 순회/다운로드)
    spans = tiles_covering_base_rect(slide, dz, lvl, x0, y0, w, h,
                                    tile_size=224, overlap=0, return_spans=True)
    print(f"row spans: {len(spans)} rows")
    print("first 5 spans:", spans[:5])  # (ty, tx0, tx1)
'''
print()

tile count: 754
first 10: [(45, 97), (46, 97), (47, 97), (48, 97), (49, 97), (50, 97), (51, 97), (52, 97), (53, 97), (54, 97)]



In [17]:
found_tiles = [
    t for t in data.get("cell_masks", [])
    if (int(t.get("x", -1)), int(t.get("y", -1))) in tiles
]
print(found_tiles[0])
print(f"총 {len(found_tiles)}개 tile dict 매칭")

{'level': 16.0, 'x': 45.0, 'y': 97.0, 'width': 224.0, 'height': 224.0, 'masks': [{'cell_id': 0.0, 'cell_type': 'Fibroblasts', 'confidence': 0.9999999973474801, 'coordinates': [[7.0, 67.0], [6.5, 67.5], [6.5, 69.0], [6.0, 69.5], [6.0, 70.5], [5.0, 71.5], [5.0, 74.5], [5.5, 75.0], [5.5, 76.0], [7.0, 77.5], [7.5, 77.5], [8.0, 78.0], [8.5, 78.0], [9.0, 78.5], [11.0, 78.5], [12.5, 77.0], [12.5, 76.0], [13.0, 75.5], [13.0, 75.0], [13.5, 74.5], [13.5, 74.0], [14.0, 73.5], [14.0, 73.0], [14.5, 72.5], [14.5, 69.5], [14.0, 69.0], [14.0, 68.5], [13.0, 67.5], [12.5, 67.5], [12.0, 67.0]], 'centroid': [9.5, 72.5]}, {'cell_id': 1.0, 'cell_type': 'Fibroblasts', 'confidence': 0.9999999927536233, 'coordinates': [[3.5, 97.0], [3.0, 97.5], [2.5, 97.5], [2.5, 98.5], [4.0, 100.0], [4.5, 100.0], [5.0, 100.5], [5.5, 100.5], [6.0, 101.0], [6.5, 101.0], [7.0, 101.5], [8.0, 101.5], [8.5, 102.0], [11.0, 102.0], [11.0, 100.0], [10.5, 99.5], [10.5, 99.0], [9.5, 98.0], [9.0, 98.0], [8.5, 97.5], [8.0, 97.5], [7.5, 97

In [18]:
# cell_type = ['Cancer cell', 'Fibroblasts', 'Macrophages', 'Lymphocytes', 'Apoptotic Body', 'Neutrophils']
roi_comment = f"ROI: x0={x0}, y0={y0}, width={w}, height={h}"
cell_type = ['a']
annot_path = f"{output_dir}/asap_{wsi_name}_{iiss[ix]}.xml"

tiles_to_asap_xml(
    wsi_path=wsi_path,
    tiles=found_tiles,                   # 3k 타일 dict 리스트
    xml_out_path=annot_path,
    color_map=COLOR_MAP,
    roi_comment = roi_comment,
    contained = list(COLOR_MAP.keys()),
    max_workers=45,                 # 코어에 맞게 조정
)

ASAP XML saved: /home/sjhong/segmentation/histoplus/output/1-028_08/asap_1-028_08_588.xml
